<a href="https://colab.research.google.com/github/mohith789p/Nvidia/blob/main/accessing_jupyter_from__jetsonNano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Accessing Jupyter from Jetson Nano
Set up a Jetson Nano with a Jupyter environment, configure Docker for USB/CSI camera access, and verify the full pipeline so you can access the Jupyter interface from a web browser on the same network.

## Introduction and Prerequisites

This guide walks you through turning a Jetson Nano into a functional development environment: reachable over SSH, running a Docker container with camera access, and exposing a Jupyter Notebook for computer-vision work. The goal is to eliminate guesswork and ensure each step is verifiable.

### Prerequisites

- **Jetson Nano with JetPack installed** and booted normally.  
- **Reliable power source** (barrel jack strongly recommended to avoid undervoltage issues).  
- **Internet connection** on the Nano for package installation and container pulls.  
- **A host PC or laptop** on the same network for SSH and browser access.  
- **A connected USB webcam or CSI camera**, attached before launching the Docker container.

Once these are in place, the remaining environment will be built inside Docker.


### Subtask
Establish an SSH connection from your laptop to the Jetson Nano.

#### Instructions

1. Identify the Jetson Nano's IP address.  
   On the Jetson terminal, run:
   ```bash
   ip addr
   ```
   Use the IP assigned to the active interface (eth0, wlan0, or usb0).

2. Open a terminal on your laptop or host computer.

3. Connect using SSH. Replace <username> and <jetson_ip>:
   ```bash
   ssh <username>@<jetson_ip>
   ```
   Example:
   ```bash
   ssh nvidia@192.168.1.100
   ```

4. Enter the Jetson's password when prompted.

5. Check the terminal prompt to confirm you are logged into the Jetson Nano.

## Create the Data Directory

### Overview

The `nvdli-data` directory is used to store datasets, project files, and any persistent data needed for your NVIDIA Docker containers. By mapping this directory to the Docker container as a volume, you can easily access and persist data between container runs.

### Instructions

1. Connect to the Jetson Nano via SSH.
2. Run the command:

   ```bash
   mkdir -p nvdli-data
   ```

## Running NVIDIA Docker with a USB or CSI Camera

### Overview

This section explains how to run an NVIDIA Docker container on Jetson Nano with access to either a USB or CSI camera. You can use the direct Docker command or create a reusable script for convenience.

### Instructions for USB Camera

1. **Connect Your USB Camera**
   Plug your USB camera into a USB port on the Jetson Nano.

2. **Identify the Camera Device**
   In the SSH terminal, list video devices:

   ```bash
   ls /dev/video*
   ```

   Typically, your camera appears as `/dev/video0`. If multiple devices exist, select the appropriate one.

3. **Run the Docker Container for USB Camera**
   Replace `<path_to_nvdli_data>` with the absolute path to your `nvdli-data` directory (e.g., `/home/nvidia/nvdli-data`) and ensure `/dev/video0` matches your camera device:

   ```bash
   sudo docker run --runtime nvidia -it --rm --network host \
       --volume <path_to_nvdli_data>:/nvdli-nano/data \
       --device /dev/video0 \
       nvcr.io/nvidia/dli/dli-nano-ai:<tag>
   ```

4. **Optional: Create a Reusable Script for USB Camera**

   ```bash
   # Create the script
   echo "sudo docker run --runtime nvidia -it --rm --network host \
       --volume ~/nvdli-data:/nvdli-nano/data \
       --device /dev/video0 \
       nvcr.io/nvidia/dli/dli-nano-ai:v2.0.2-r32.7.1" > docker_dli_run_usb.sh

   # Make it executable
   chmod +x docker_dli_run_usb.sh

   # Run the script
   ./docker_dli_run_usb.sh
   ```

### Instructions for CSI Camera

1. **Connect Your CSI Camera**
   Ensure your CSI camera is properly connected to the Jetson Nano's CSI port. CSI cameras do not have a standard device file like USB cameras and are accessed through the NVIDIA Argus API.

2. **Run the Docker Container for CSI Camera**
   Replace `<path_to_nvdli_data>` with the absolute path to your `nvdli-data` directory:

   ```bash
   # Create a reusable script for CSI camera
   echo "sudo docker run --runtime nvidia -it --rm --network host \
       --volume ~/nvdli-data:/nvdli-nano/data \
       --volume /tmp/argus_socket:/tmp/argus_socket \
       --device /dev/nvargus-daemon \
       nvcr.io/nvidia/dli/dli-nano-ai:v2.0.2-r32.7.1" > docker_dli_run_csi.sh

   # Make it executable
   chmod +x docker_dli_run_csi.sh

   # Run the script
   ./docker_dli_run_csi.sh
   ```

**Key Difference for CSI Camera:** The `--device /dev/nvargus-daemon` argument is used instead of `/dev/video0`, granting Docker access to the CSI camera through the NVIDIA Argus Camera daemon.

## Verify Docker Container and Files

### Overview

After starting the Docker container, it is important to verify that the container is running correctly and that the mapped data directory contains the expected files. This ensures your workspace inside the container is properly set up.

### Instructions

1. List the contents of the current directory to verify files:

    ```bash
    ls
    ```

## Access Jupyter Notebook

### Overview

After running the NVIDIA Docker container, you can access the Jupyter Notebook interface from a browser on your host computer. This allows you to interactively run Python code, manage your projects, and view results directly from the Jetson Nano.

### Instructions

1. Open a web browser on your host computer.
2. In the address bar, type the following URL, replacing `<jetson_nano_ip_address>` with your Jetson Nano's IP address:

   ```
   http://<jetson_nano_ip_address>:8888
   ```
3. When prompted, enter the default password:

   ```
   dlinano
   ```
4. You should now see the Jupyter Notebook dashboard. It is recommended to change the password after your first login for security purposes.

# Summary

In this notebook, we set up the nvdli-data directory for persistent storage, ran NVIDIA Docker containers for both USB and CSI cameras with optional reusable scripts and accessed the Jupyter Notebook interface for interactive development.